<a href="https://colab.research.google.com/github/JonathanSum/TorchAudio_and_TorchTextNotes/blob/main/Attention_Transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt

     |████████████████████████████████| 24.2 MB 68.1 MB/s 
     |████████████████████████████████| 1.4 MB 58.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
     |████████████████████████████████| 50.4 MB 20 kB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 1.9 MB 71.9 MB/s 
     |████████████████████████████████| 7.1 MB 65.6 MB/s 
     |████████████████████████████████| 17.4 MB 209 kB/s 
     |████████████████████████████████| 1.9 MB 56.7 MB/s 
     |████████████████████████████████| 895 kB 67.4 MB/s 
     |████████████████████████████████| 3.3 MB 56.7 MB/s 
  Created wheel for nltk: filename=nltk-3.5-py3-none-any.whl size=1434688 sha256=d30177ebb999654c91adac243c8430abfb2ebe7d621c0721caeadff116218895
  Stored in directory: /root/.cache/pip/wheels/45/6c/46/a1865e7ba706b3817f5d1b2ff7ce8996aabdd0d03d47ba0266
Successfully built nltk
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Unin

In [4]:
!wget -q https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/torchnlp.py

In [5]:
import torch
import torchtext
from torchnlp import *
import transformers
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_len = len(vocab)

Loading dataset...


train.csv: 29.5MB [00:00, 89.7MB/s]
test.csv: 1.86MB [00:00, 44.8MB/s]                  


Building vocab...


In [6]:
# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
# bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

In [8]:
PAD_INDEX, UNK_INDEX

(0, 100)

In [11]:
tokenizer.encode('PyTorch is a great framework for NLP')

[101, 1052, 22123, 2953, 2818, 2003, 102]

In [12]:
def pad_bert(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [tokenizer.encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0] for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, collate_fn=pad_bert, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, collate_fn=pad_bert)

In [13]:
model = transformers.BertForSequenceClassification.from_pretrained(bert_model,num_labels=4).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

report_freq = 50
iterations = 50000 # make this larger to train for longer time!

model.train()

i,c = 0,0
acc_loss = 0
acc_acc = 0

for labels,texts in train_loader:
    labels = labels.to(device)-1 # get labels in the range 0-3         
    texts = texts.to(device)
    loss, out = model(texts, labels=labels)[:2]
    labs = out.argmax(dim=1)
    acc = torch.mean((labs==labels).type(torch.float32))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    acc_loss += loss
    acc_acc += acc
    i+=1
    c+=1
    if i%report_freq==0:
        print(f"Loss = {acc_loss.item()/c}, Accuracy = {acc_acc.item()/c}")
        c = 0
        acc_loss = 0
        acc_acc = 0
    iterations-=1
    if not iterations:
        break

Loss = 0.32638694763183596, Accuracy = 0.8925
Loss = 0.32102485656738283, Accuracy = 0.8925
Loss = 0.36320201873779295, Accuracy = 0.8875
Loss = 0.3732812118530273, Accuracy = 0.875
Loss = 0.3281418228149414, Accuracy = 0.89
Loss = 0.305573844909668, Accuracy = 0.895
Loss = 0.4528038024902344, Accuracy = 0.8375
Loss = 0.27932039260864255, Accuracy = 0.905
Loss = 0.4075177001953125, Accuracy = 0.8725
Loss = 0.32272430419921877, Accuracy = 0.905
Loss = 0.3305033111572266, Accuracy = 0.8875
Loss = 0.35128997802734374, Accuracy = 0.875
Loss = 0.34546363830566407, Accuracy = 0.885
Loss = 0.3207216262817383, Accuracy = 0.885
Loss = 0.26595766067504883, Accuracy = 0.905
Loss = 0.3073245048522949, Accuracy = 0.91
Loss = 0.2440859031677246, Accuracy = 0.9225
Loss = 0.39716693878173825, Accuracy = 0.86
Loss = 0.26120748519897463, Accuracy = 0.9175
Loss = 0.2497698211669922, Accuracy = 0.92
Loss = 0.2609954643249512, Accuracy = 0.91
Loss = 0.30839933395385744, Accuracy = 0.89
Loss = 0.36793083190